# 导入需要的库

In [1]:
import os
import asyncio
import requests

from dotenv import load_dotenv
from typing import Annotated

from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

In [2]:
load_dotenv()

ai_agent_settings = AzureAIAgentSettings.create()

SERP_API_KEY = os.getenv("SERPAPI_SEARCH_API_KEY")

BASE_URL = 'https://serpapi.com/search?engine=bing'

In [3]:
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="Booking hotel.")
    def booking_hotel(self, 
                      query: Annotated[str, "The name of city."],
                      check_in_date: Annotated[str, "Hotel check-in date."],
                      check_out_date: Annotated[str, "Hotel check-out date."]) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of city
        - check_in_date: Hotel check-in date
        - check_out_date: Hotel check-out date
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adult": "1",
            "currency": "CNY",
            "gl": "cn",
            "hl": "zh-cn",
            "api_key": SERP_API_KEY
        }

        response = requests.get(BASE_URL, params=params)

        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None
        
    @kernel_function(description="Booking flight.")
    def booking_flight(self,
                       origin: Annotated[str, "The name of Departure."],
                       destination: Annotated[str, "The name of Destination."],
                       outbound_date: Annotated[str, "The data of outbound."],
                       return_date: Annotated[str, "The data of return."]) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The data of outbound
        - return_date: The data of return
        Returns:
        - The result of booking flight information
        """

        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "CNY",
            "hl": "zh-cn",
            "gl": "cn",
            "adults": "1",
            "type": "1",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }

        go_response = requests.get(BASE_URL, params=go_params)

        result = ""

        if go_response.status_code == 200:
            response = go_response.json()
            result += "# outbound \n " + str(response)

        else:
            print("error!!!")

        # Define the parameters for the return flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "CNY",
            "hl": "zh-cn",
            "gl": "cn",
            "adults": "1",
            "type": "1",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }

        back_response = requests.get(BASE_URL, params=go_params)
        if back_response.status_code == 200:
            response = back_response.json()
            result += "# return \n " + str(response)
        else:
            print("error!!!")

        print(result)

        return result
